In [2]:
import os
import sys
import pandas as pd
import numpy as np
import sklearn
import nltk
from nltk.tokenize import word_tokenize
import random as rnd
from sklearn.feature_extraction.text import TfidfVectorizer
from os.path import join
from jpype import JClass, getDefaultJVMPath, java, shutdownJVM, startJVM
import nltk.data
import codecs
    
if __name__ == '__main__':

    ZEMBEREK_PATH: str = join('..', '..', 'bin', 'zemberek-full.jar')

    startJVM(
            getDefaultJVMPath(),
            '-ea',
            f'-Djava.class.path={"C://Users//ASUS//Desktop//cz//distributions//0.17.1//zemberek-full.jar"}',
            convertStrings=False
    )
   
    TurkishMorphology: JClass = JClass('zemberek.morphology.TurkishMorphology')
    Paths: JClass = JClass('java.nio.file.Paths')

    morphology: TurkishMorphology = TurkishMorphology.createWithDefaults()

path = "C://Users//ASUS//Desktop//cz//1150haberAZ"
fileNames = []
wordList = []

fileNames = [ subdir+os.path.sep+file for subdir, dirs, files in os.walk(path) for file in files ]

tfidfVectorizer = TfidfVectorizer(decode_error='ignore')
docTermMatrix = tfidfVectorizer.fit_transform((open(f,encoding="utf8").read() for f in fileNames))

wordList = [ word[0] for word in tfidfVectorizer.vocabulary_.items() ]
verb_list = []

for f in fileNames:
    doc = codecs.open(f, 'r', 'utf-8')
    content = doc.read()

    tokenizer = nltk.data.load('tokenizers/punkt/turkish.pickle')
    tokenized_sentence = tokenizer.tokenize(content)
    

java.lang.NoClassDefFoundError: zemberek/morphology/TurkishMorphology

In [3]:
from bs4 import BeautifulSoup
import urllib
from urllib.request import urlopen
import re
import string
import os
from os.path import join
from jpype import JClass, JString, getDefaultJVMPath, java, shutdownJVM, startJVM


def find_synonyms(sentence):   
    global new_sentence
    sentence = sentence.translate(str.maketrans('', '', string.punctuation))
#     print("ORIGINAL SENTENCE: " + sentence)
    return_sentence = ""
    sentence_words = []
    sentence_for_synonym_search = []
    
    if __name__ == '__main__':

        ZEMBEREK_PATH: str = join('..', '..', 'bin', 'zemberek-full.jar')

        TurkishMorphology: JClass = JClass('zemberek.morphology.TurkishMorphology')
        Paths: JClass = JClass('java.nio.file.Paths')

        morphology: TurkishMorphology = TurkishMorphology.createWithDefaults()
    
    for i in sentence.split():
        sentence: str = i
        analysis: java.util.ArrayList = morphology.analyzeSentence(sentence)

        results: java.util.ArrayList = (
            morphology.disambiguate(sentence, analysis).bestAnalysis()
        )
        
        #add sentence just subject, adjective, noun and verb from original sentence
        for j, result in enumerate(results, start=0):
            x = (str(result)).split(":")[1]
            y = x.split("]")[0]
            if y == "Pron,Pers": #Select Subject Data
                x = (str(result)).split(":")[0]
                y = x.split("[")[1]
                sentence_for_synonym_search.append(y)
            elif y == "Adj": #Select Adj Data
                x = (str(result)).split(":")[0]
                y = x.split("[")[1]
                sentence_words.append(sentence.split()[j])
                sentence_for_synonym_search.append(y)
            elif y == "Noun": #Select Noun Data
                x = (str(result)).split(":")[0]
                y = x.split("[")[1]
                sentence_words.append(sentence.split()[j])
                sentence_for_synonym_search.append(y)
            elif y == "Verb": #Select Verb Data
                x = (str(result)).split(":")[0]
                y = x.split("[")[1]
                sentence_words.append(sentence.split()[j])
                sentence_for_synonym_search.append(y)

    word_new = ""
    
    for kelime,original_word in zip(sentence_for_synonym_search,sentence_words):
        kelime_new = kelime.replace('ö','o')
        kelime_new = kelime_new.replace('ı','i')
        kelime_new = kelime_new.replace('ü','u')
        kelime_new = kelime_new.replace('ç','c')
        kelime_new = kelime_new.replace('ş','s')
        kelime_new = kelime_new.replace('ğ','g')
        kelime_new = kelime_new.strip()
#         print("KELIME_NEW: " + kelime_new)
        #find synonyms from site
#         try:

        if kelime not in verb_list:
            user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'

            url = "http://www.es-anlam.com/kelime/" + kelime_new
            headers={'User-Agent':user_agent,} 

            request=urllib.request.Request(url,None,headers) #The assembled request
            response = urllib.request.urlopen(request)
            soup = BeautifulSoup(response, "html.parser")

            ana = soup.find('h2')
            alt=ana.find('strong')
            alt = str(alt)
            alt = (alt.split(">")[1]).split(",")[0]
            alt = alt.split("<")[0]
            alt = alt.strip()

            if alt == "BULUNAMADI !":
                new_sentence = new_sentence + " " + original_word
            else:
#                 new_sentence = new_sentence + " " + alt
                word_new = alt

                add_suffix(original_word, word_new)   
#             print("NEW_SENTENCE IF: " + new_sentence)
    
        else:
            new_sentence += " " + original_word
#             print("NEW_SENTENCE ELSE: " + new_sentence)

    return new_sentence
    

def add_suffix(sentence_word, synonym_word):
    global new_sentence
    TurkishMorphology: JClass = JClass('zemberek.morphology.TurkishMorphology')
    WordAnalysis: JClass = JClass('zemberek.morphology.analysis.WordAnalysis')

    morphology: TurkishMorphology = TurkishMorphology.createWithDefaults()
    
    results: WordAnalysis = morphology.analyze(JString(sentence_word))

    number = ""
    possessive = ""
    case = ""
    word = ""

    for result in results:
        result = str(result)
        try:#set number
            if "A3sg" in result:
                number = "A3sg"
            elif "A3pl" in result:
                number = "A3pl"
        except:
            number = ""
        
        try: #set possessive
            if "P1sg" in result:
                possessive ="P1sg"
            elif "P2sg" in results:
                possessive = "P2sg"
            elif "P3sg" in result:
                possessive = "P3sg"
        except:
            possessive = ""
        
        try: #set case
            if "Dat" in result:
                case = "Dat"
            elif "Loc" in result:
                case = "Loc"
            elif "Abl" in result:
                case = "Abl"
        except:
            case = ""   #cases: List[JString] = [JString('Dat'), JString('Loc'), JString('Abl')]
        
        word = result.split(" ")[1]
        word = word.split(":")[0]

    morphology: TurkishMorphology = (
        TurkishMorphology.builder().setLexicon(synonym_word).disableCache().build()
    )

    try:
        item = morphology.getLexicon().getMatchingItems(synonym_word).get(0)

        if number != "" and possessive != "" and case != "":
            for result in morphology.getWordGenerator().generate(item, number, possessive, case):
                new_sentence += " " + str(result.surface)
        elif number != "" and possessive != "" and case == "":
             for result in morphology.getWordGenerator().generate(item, number, possessive):
                new_sentence += " " + str(result.surface)
        elif number != "" and possessive == "" and case != "":
             for result in morphology.getWordGenerator().generate(item, number, case):
                new_sentence += " " + str(result.surface)
        elif number == "" and possessive != "" and case != "":
             for result in morphology.getWordGenerator().generate(item, possessive, case):
                new_sentence += " " + str(result.surface)
        elif number == "" and possessive == "" and case != "":
             for result in morphology.getWordGenerator().generate(item, case):
                new_sentence += " " + str(result.surface)
        elif number == "" and possessive != "" and case == "":
             for result in morphology.getWordGenerator().generate(item, possessive):
                new_sentence += " " + str(result.surface)
        elif number != "" and possessive == "" and case == "":
             for result in morphology.getWordGenerator().generate(item, number):
                new_sentence += " " + str(result.surface)
        else:
            new_sentence += str(result.surface)
    except:
        pass

In [ ]:
def valueList(word_list):
    
    letterValue = {'a':1, 'b':2, 'c':3, 'ç':4, 'd':5, 'e':6, 'f':7, 'g':8, 'ğ':9, 'h':10, 'ı':11,
                 'i':12, 'j':13, 'k':14, 'l':15, 'm':16, 'n':17, 'o':18, 'ö':19, 'p':20, 'r':21,
                 's':22, 'ş':23, 't':24, 'u':25, 'ü':26, 'v':27, 'y':28, 'z':29 }
    values = []
    
    for word in word_list:
        total = 0      
        for letter in word:
            if letter not in letterValue:
                continue
            total = total + letterValue[letter]
            
        values.append(total)

    return values

In [1]:
new_sentence = "" #global sentence for generating new sentence

def generate_sentence():
    global verb_list
    global new_sentence
    sentence_sum = 600 #SENTENCE VALUE
    
    for i in wordList:
        sentence: str = i
        analysis: java.util.ArrayList = morphology.analyzeSentence(sentence)

        results: java.util.ArrayList = (
            morphology.disambiguate(sentence, analysis).bestAnalysis()
        )
        
        for j, result in enumerate(results, start=0):
            x = (str(result)).split(":")[1]
            y = x.split("]")[0]

            if y == "Verb": #Select Verb Data
                x = (str(result)).split(":")[0]
                y = x.split("[")[1]
                verb_list.append(y)
                verb_list = list(dict.fromkeys(verb_list))

    gen_sentence_list = []
    sentence_value = 0
    cntrl = 0

    print("GENERATING SENTENCE, PLEASE WAIT...")
    for sentence in tokenized_sentence:
        if cntrl == 1:
            new_sentence = ""
        synonym_sentence = ""
        synonym_sentence = find_synonyms(sentence)
        synonym_sentence = synonym_sentence[1:]
        new_sentence = synonym_sentence.capitalize()
#         print("NEW SENTENCE: " + new_sentence)
        cntrl = 1
        
        sentence_value = valueList(synonym_sentence)
        
        total_value = 0
        for value in sentence_value:
            total_value += value
            
        if total_value <= sentence_sum+100 and total_value > sentence_sum:
            gen_sentence_list.append(new_sentence)
            
        if len(gen_sentence_list) == 1:
            break
               
    for i in gen_sentence_list:
        print(i)   
    
generate_sentence()
shutdownJVM()

NameError: name 'wordList' is not defined